In [56]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from datetime import datetime as dt
from torch import nn

In [57]:
# data
trans = transforms.ToTensor()
data_train = torchvision.datasets.FashionMNIST(root='../data', train=True, transform=trans)
data_test = torchvision.datasets.FashionMNIST(root='../data', train=False, transform=trans)

print(len(data_train), len(data_test))
print(data_train[0][0].shape, data_train[0][1])

size_batch = 256
iter_train = data.DataLoader(data_train, size_batch, shuffle=True, num_workers=2)
iter_test = data.DataLoader(data_test, size_batch, shuffle=False, num_workers=2)

t0 = dt.now()
for X_batch, y_batch in iter_train:
    continue

print(dt.now() - t0)
t0 = dt.now()
for X_batch, y_batch in iter_test:
    continue

print(dt.now() - t0)

60000 10000
torch.Size([1, 28, 28]) 9
0:00:02.957989
0:00:01.253965


In [58]:
# model
num_in = 784
num_out = 10
W_model = torch.normal(0, 0.01, size=(num_in, num_out), requires_grad=True)
b_model = torch.zeros(num_out, requires_grad=True)
def softmax(X):
    X_exp = torch.exp(X)
    denom = X_exp.sum(1, keepdim=True)
    return X_exp / denom

def model(X):
    return softmax(torch.matmul(X.reshape([-1, num_in]), W_model) + b_model)

def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

# evaluate
def accuracy(y_hat, y):
    y_hat = y_hat.argmax(axis=1)
    return float((y_hat.type(y.dtype) == y).sum())

def evaluate_accuracy(md, iter_data):
    if isinstance(md, torch.nn.Module):
        md.eval()

    metric = [0.0, 0.0]
    for X, y in iter_data:
        metric[0] += accuracy(md(X), y)
        metric[1] += y.numel()

    return metric[0] / metric[1]

print(evaluate_accuracy(model, iter_test))

# abandon

0.1212


In [59]:
# parameters
lr_learn = 0.1
epoch_learn = 10

# model
num_in = 784
num_out = 10
model = nn.Sequential(nn.Flatten(), nn.Linear(num_in, num_out))
# def init_weights(md):
#     if type(md) == nn.Linear:
#         nn.init.normal_(md.weight, std=0.01)
#
# model.apply(init_weights)
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(model.parameters(), lr=lr_learn)

# training
for epoch in range(epoch_learn):
    for X_batch, y_batch in iter_train:
        loss(model(X_batch), y_batch).backward()
        trainer.step()
        trainer.zero_grad()

    p_acc = evaluate_accuracy(model, iter_test)
    print('epoch {}, p-acc {:.4f}'.format(epoch + 1, p_acc))

epoch 1, p-acc 0.7754
epoch 2, p-acc 0.7987
epoch 3, p-acc 0.8168
epoch 4, p-acc 0.8183
epoch 5, p-acc 0.8266
epoch 6, p-acc 0.8267
epoch 7, p-acc 0.8262
epoch 8, p-acc 0.8296
epoch 9, p-acc 0.8313
epoch 10, p-acc 0.8206
